# Demo Notebook
Instruction on how to use models to query recommendation.

### Dependencies

In [13]:
from models import get_recommendation
from models.cb import FeatureGenerator
from models.cf import EmbededRatingNet
from models.hybrid import EmbededHybridNet
from utility import Mapper

import torch
import pandas as pd
from typing import Iterable
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {DEVICE}")

Using cuda


### Import movies dataset and mapper

In [5]:
metadatas = pd.read_csv("./database/merged/metadatas.csv")
movie_ratings = pd.read_csv("./database/merged/global_ratings.csv")
movies = pd.merge(metadatas, movie_ratings, on="MovieID")
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87585 entries, 0 to 87584
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   MovieID          87585 non-null  int64  
 1   Title            87585 non-null  object 
 2   Runtime          87530 non-null  object 
 3   ReleaseDate      87577 non-null  object 
 4   Genres           87585 non-null  object 
 5   Directors        87585 non-null  object 
 6   Cast             87585 non-null  object 
 7   OriginCountries  87585 non-null  object 
 8   Languages        87585 non-null  object 
 9   Plot             87389 non-null  object 
 10  AverageRating    87585 non-null  float64
 11  VoteCount        87585 non-null  float64
dtypes: float64(2), int64(1), object(9)
memory usage: 8.0+ MB


In [9]:
# Mapper for cf since not all movies are rated
CF_MAPPER = Mapper.load("./database/merged/full/pydata/mapper.pkl")
# Mapper for cb and hybrid, which includes all movies
FULL_MAPPER = Mapper.load("./database/merged/mapper.pkl")

### Query using indices

In [17]:
def get_query(movie_data: pd.DataFrame, mapper: Mapper, indices: Iterable[int]):
    movie_ids = [mapper.item_inv_map[int(i)] for i in indices]
    return movie_data[movie_data["MovieID"].isin(movie_ids)]

### Recommedation using CB

Import model

In [34]:
tf_mat = torch.load("./models/cb/directors_and_cast.pt")
recommender = get_recommendation("cb", 10, DEVICE)
cb = FeatureGenerator(tf_mat)

**Input** : Movie index (not id, must use mapper) \
**Return**: Movies indices and similarity scores

In [67]:
movie_id = 1
movie_idx = FULL_MAPPER.item_fwd_map[movie_id]
movie_title = str(movies[movies["MovieID"] == movie_id]["Title"].values[0])
idxs, sims = recommender(movie_idx, cb)
print("Movie idices:", idxs.numpy())
print("Similarity:", sims.numpy())

Movie idices: [ 3021 14810 43437 24845   715  3284  8900 18289 22171 22884]
Similarity: [0.7406386  0.66156566 0.58187306 0.57222533 0.5556421  0.5556421
 0.5556421  0.5556421  0.5556421  0.5556421 ]


In [69]:
print("Since you watched:", movie_title)
get_query(movies, FULL_MAPPER, idxs)

Since you watched: Toy Story (1995)


,MovieID,Title,Runtime,ReleaseDate,Genres,Directors,Cast,OriginCountries,Languages,Plot,AverageRating,VoteCount
715,730,Low Life (1994),NaN,1994-01-01,['Drama'],"[""John L'Ecuyer""]",[],['Canada'],[],NaN,3.057143,70.0
3021,3114,Toy Story 2 (1999),01:32:00,1999-10-30,"['Adventure', 'Animation', 'Children', 'Comedy...","['John Lasseter', 'Ash Brannon', 'Lee Unkrich']","['Tom Hanks', 'Tim Allen', 'Joan Cusack', 'Kel...",['United States'],['English'],"When Woody is stolen by a toy collector, Buzz ...",3.942756,679463.0
3284,3378,"Ogre, The (Unhold, Der) (1996)",01:58:00,1996-09-12,['Drama'],['Volker Schlöndorff'],"['John Malkovich', 'Armin Mueller-Stahl', 'Got...","['Germany', 'France', 'United Kingdom', 'Poland']","['English', 'German', 'French', 'Hebrew', 'Lat...","Frenchman Abel Tiffauges likes children, and w...",3.395856,2809.0
8900,26602,Pathfinder (Ofelas) (1987),01:26:00,1987-09-03,"['Action', 'Adventure', 'Drama']",['Nils Gaup'],"['Mikkel Gaup', 'Ingvald Guttorm', 'Nils Utsi'...",['Norway'],"['Saami', 'Northern Sami']",A young Sámi man witnesses a band of vicious r...,3.600112,5820.0
14815,78499,Toy Story 3 (2010),01:43:00,2010-06-16,"['Adventure', 'Animation', 'Children', 'Comedy...",['Lee Unkrich'],"['Tom Hanks', 'Tim Allen', 'Joan Cusack', 'Ned...",['United States'],"['English', 'Spanish']",The toys are mistakenly delivered to a day-car...,4.140921,946928.0
18314,95856,Knick Knack (1989),00:04:00,1989-11-23,"['Animation', 'Children']",['John Lasseter'],[],['United States'],"['English', 'Hebrew']",A snow globe snowman wants to join the other t...,3.693774,16904.0
22294,114197,Noam Chomsky: Distorted Morality (2003),00:55:00,2003-01-01,['Documentary'],['John Junkerman'],['Noam Chomsky'],['United States'],['English'],"The noted scholar, political and social analys...",3.927011,537.0
23044,116837,The Little Fox (1981),01:19:00,1981-12-10,"['Adventure', 'Animation', 'Children']",['Attila Dargay'],"['Judit Pogány', 'József Gyabronka', 'Teri Föl...",['Hungary'],"['Hungarian', 'French', 'German']","Famous, great hunters, better stay away, here ...",4.142422,6192.0
25234,123200,Jim Jefferies: I Swear to God (2009),00:58:00,2009-05-16,['Comedy'],['John Moffitt'],['Jim Jefferies'],['United States'],['English'],Politically incorrect Australian comedian Jim ...,4.031740,3271.0
45459,169318,Theo Von: No Offense (2016),01:07:00,2016-02-26,['Comedy'],['John Asher'],['Theo Von'],['United States'],['English'],Southern comedian Theo Von offers up a twisted...,2.906664,1133.0


### Recommedation using CF

Import model

In [75]:
num_users = len(CF_MAPPER.user_fwd_map)
num_items = len(CF_MAPPER.item_fwd_map)
cf = EmbededRatingNet(num_users, num_items, 8)
cf.load_state_dict(torch.load("./models/cf/embedded_rating_8.pth"))
recommender = get_recommendation("cf", 10, DEVICE)

**Input** : User index (not id, must use mapper) \
**Return**: Movies indices and predicted ratings

In [76]:
user_id = 123
user_idx = CF_MAPPER.user_fwd_map[user_id]
idxs, ratings = recommender(user_idx, num_items, cf)
print("Movie indices:", idxs.numpy())
print("Predicted ratings:", ratings.numpy())

Movie indices: [66936 77954 78386  2908 21607 70511 44473 81177 51040 69619]
Predicted ratings: [0.74531263 0.7442857  0.7433104  0.74181724 0.7407524  0.73996395
 0.739172   0.7388143  0.7384881  0.7381683 ]


In [77]:
print("Recommedation for user", user_id)
get_query(movies, CF_MAPPER, idxs)

Recommedation for user 123


,MovieID,Title,Runtime,ReleaseDate,Genres,Directors,Cast,OriginCountries,Languages,Plot,AverageRating,VoteCount
2908,3000,Princess Mononoke (Mononoke-hime) (1997),02:13:00,1997-07-12,"['Action', 'Adventure', 'Animation', 'Drama', ...",['Hayao Miyazaki'],"['Yôji Matsuda', 'Yuriko Ishida', 'Yûko Tanaka...",['Japan'],['Japanese'],On a journey to find the cure for a Tatarigami...,4.151970,471709.0
21700,111759,Edge of Tomorrow (2014),01:53:00,2014-05-27,"['Action', 'Sci-Fi', 'IMAX']",['Doug Liman'],"['Tom Cruise', 'Emily Blunt', 'Bill Paxton', '...","['United States', 'Canada', 'India']",['English'],A soldier fighting aliens gets to relive the s...,3.949683,790512.0
46544,171611,Baahubali 2: The Conclusion (2017),02:47:00,2017-04-27,"['Action', 'Adventure', 'War']",['S.S. Rajamouli'],"['Prabhas', 'Rana Daggubati', 'Anushka Shetty'...",['India'],"['Telugu', 'Tamil']","Amarendra Baahubali, the heir apparent to the ...",4.097670,119884.0
53332,185841,Heavy Trip (2018),01:32:00,2018-03-09,['Comedy'],"['Juuso Laatio', 'Jukka Vidgren']","['Johannes Holopainen', 'Samuli Jaskio', 'Antt...","['Finland', 'Norway', 'Belgium']","['Finnish', 'English', 'Norwegian']","After years of playing songs by other artists,...",3.499772,11273.0
69479,223760,Little Red Riding Hood (1917),00:07:00,1917-05-03,['Animation'],[],[],['United States'],"['No Language', 'English']",A silhouette retelling the fairy tale,3.000000,1.0
72270,233899,Midway to Love (2019),01:27:00,2019-06-01,['Romance'],['Jeffrey Day'],"['Rachel Hendrix', 'Daniel Stine', 'Andrew Hun...",['United States'],"['English', 'French']",When the network announces that her television...,2.804230,643.0
73280,238050,Mercy Mission: The Rescue of Flight 771 (1993),01:35:00,1993-12-13,['Drama'],['Roger Young'],"['Robert Loggia', 'Scott Bakula', 'Rebecca Rig...","['Australia', 'United States']",['English'],"A pilot ferrying a crop duster aircraft, from ...",3.236690,654.0
81042,272211,Ambulance (2022),02:16:00,2022-03-16,"['Action', 'Crime', 'Thriller']",['Michael Bay'],"['Jake Gyllenhaal', 'Yahya Abdul-Mateen II', '...","['United States', 'Japan']","['English', 'Spanish', 'Cantonese']",Two robbers steal an ambulance after their hei...,3.058375,107334.0
81476,273715,Help Me... I'm Possessed (1974),01:19:00,1974-10-25,['Horror'],['Charles Nizet'],"['Bill Greer', 'Deedy Peters', 'Lynne Marta', ...",['United States'],['English'],"A mad doctor runs a sanitarium in the desert, ...",2.108699,246.0
84312,281848,Fly Out (2009),01:56:00,2009-07-15,"['Comedy', 'Fantasy', 'Romance']",['Ahmed al-Gendy'],"['Ahmed Mekky', 'Donia Samir Ghanem', 'Maged E...",['Egypt'],['Arabic'],A nerdy vet gets help from a genie to catch th...,3.649988,4434.0


### Recommedation using Hybrid (similar to CF)

Import model

In [79]:
num_users = len(FULL_MAPPER.user_fwd_map)
num_items = len(FULL_MAPPER.item_fwd_map)
hybrid = EmbededHybridNet(
    num_users,
    genres=torch.load("./models/cb/genres_with_ratings.pt"),
    titles_and_plots=torch.load("./models/cb/titles_and_plots.pt"),
    directors_and_casts=torch.load("./models/cb/directors_and_cast.pt"),
)
hybrid.load_state_dict(torch.load("./models/hybrid/embedded_hybrid.pth"))
recommender = get_recommendation("cf", 10, DEVICE)

**Input** : User index (not id, must use mapper) \
**Return**: Movies indices and predicted ratings

In [80]:
user_id = 123
user_idx = FULL_MAPPER.user_fwd_map[user_id]
idxs, ratings = recommender(user_idx, num_items, hybrid)
print("Movie indices:", idxs.numpy())
print("Predicted ratings:", ratings.numpy())

Movie indices: [39306 44190   314   840 44042  1190   522  1173   292   351]
Predicted ratings: [0.955963   0.94663024 0.9440476  0.9353055  0.92057014 0.9159558
 0.9126867  0.9108865  0.9004415  0.89932597]


In [81]:
print("Recommedation for user", user_id)
get_query(movies, FULL_MAPPER, idxs)

,MovieID,Title,Runtime,ReleaseDate,Genres,Directors,Cast,OriginCountries,Languages,Plot,AverageRating,VoteCount
292,296,Pulp Fiction (1994),02:34:00,1994-09-10,"['Comedy', 'Crime', 'Drama', 'Thriller']",['Quentin Tarantino'],"['John Travolta', 'Uma Thurman', 'Samuel L. Ja...",['United States'],"['English', 'Spanish', 'French']","The lives of two mob hitmen, a boxer, a gangst...",4.438301,2426280.0
314,318,"Shawshank Redemption, The (1994)",02:22:00,1994-09-23,"['Crime', 'Drama']",['Frank Darabont'],"['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'...",['United States'],['English'],A banker convicted of uxoricide forms a friend...,4.639923,3130026.0
351,356,Forrest Gump (1994),02:22:00,1994-06-23,"['Comedy', 'Drama', 'Romance', 'War']",['Robert Zemeckis'],"['Tom Hanks', 'Robin Wright', 'Gary Sinise', '...",['United States'],['English'],The history of the United States from the 1950...,4.384807,2427701.0
522,527,Schindler's List (1993),03:15:00,1993-12-15,"['Drama', 'War']",['Steven Spielberg'],"['Liam Neeson', 'Ralph Fiennes', 'Ben Kingsley...",['United States'],"['English', 'Hebrew', 'German', 'Polish', 'Lat...","In German-occupied Poland during World War II,...",4.486331,1589675.0
840,858,"Godfather, The (1972)",02:55:00,1972-03-14,"['Crime', 'Drama']",['Francis Ford Coppola'],"['Marlon Brando', 'Al Pacino', 'James Caan', '...",['United States'],"['English', 'Italian', 'Latin']",The aging patriarch of an organized crime dyna...,4.589663,2187022.0
1173,1203,12 Angry Men (1957),01:36:00,1957-04-10,['Drama'],['Sidney Lumet'],"['Henry Fonda', 'Lee J. Cobb', 'Martin Balsam'...",['United States'],['English'],The jury in a New York City murder trial is fr...,4.493002,923491.0
1190,1221,"Godfather: Part II, The (1974)",03:22:00,1974-12-18,"['Crime', 'Drama']",['Francis Ford Coppola'],"['Al Pacino', 'Robert De Niro', 'Robert Duvall...",['United States'],"['English', 'Italian', 'Spanish', 'Latin', 'Si...",The early life and career of Vito Corleone in ...,4.491915,1455526.0
40985,159817,Planet Earth (2006),00:50:00,2007-03-25,['Documentary'],"['Sigourney Weaver', 'David Attenborough', 'Ni...","['Sigourney Weaver', 'David Attenborough', 'Ni...","['United Kingdom', 'Canada', 'United States', ...",['English'],A documentary series on the wildlife found on ...,4.696679,226948.0
46103,170705,Band of Brothers (2001),01:00:00,2001-09-09,"['Action', 'Drama', 'War']","['Scott Grimes', 'Damian Lewis', 'Ron Livingst...","['Scott Grimes', 'Damian Lewis', 'Ron Livingst...","['United Kingdom', 'United States']","['English', 'Dutch', 'French', 'German', 'Lith...",The story of Easy Company of the U.S. Army 101...,4.698599,548811.0
46252,171011,Planet Earth II (2016),00:50:00,2017-02-18,['Documentary'],"['David Attenborough', 'Gordon Buchanan', 'Cha...","['David Attenborough', 'Gordon Buchanan', 'Cha...","['United Kingdom', 'Germany', 'France', 'China...","['English', 'French']",David Attenborough returns with a new wildlife...,4.746383,163956.0
